<a href="https://colab.research.google.com/github/j-ranasinghe/Spam-E-mail-Detection/blob/main/Spam_E_mail_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

# Importing the dataset

In [ ]:
cols = ['capital_run_length_total','capital_run_length_longest','capital_run_length_average','char_freq_#','char_freq_$','char_freq_!','char_freq_[','char_freq_(','char_freq_;','word_freq_conference','word_freq_table','word_freq_edu','word_freq_re','word_freq_project','word_freq_original','word_freq_meeting','word_freq_cs','word_freq_direct','word_freq_pm','word_freq_parts','word_freq_1999','word_freq_technology','word_freq_85','word_freq_415','word_freq_data','word_freq_857','word_freq_telnet','word_freq_labs','word_freq_lab','word_freq_650','word_freq_george','word_freq_hpl','word_freq_hp','word_freq_money','word_freq_000','word_freq_font','word_freq_your','word_freq_credit','word_freq_you','word_freq_email','word_freq_business','word_freq_free','word_freq_addresses','word_freq_report','word_freq_people','word_freq_will','word_freq_receive','word_freq_mail','word_freq_order','word_freq_internet','word_freq_remove','word_freq_over','word_freq_our','word_freq_3d','word_freq_all','word_freq_address','word_freq_make','class']
df = pd.read_csv('spambase.data' , names= cols )
df.head()

In [ ]:
df.head(20)

In [ ]:
df.tail(10)

# Data Visualisation

In [ ]:
df.dtypes #Check the data type of the attributes


In [ ]:
df.shape #Check the shape of the dataset

In [ ]:
#Check the distribution of the spam vs not-spam 
new_class = pd.Categorical(df["class"])
new_class = new_class.rename_categories(["spam","not_spam"])              

new_class.describe()

In [ ]:
df.describe() #Get the descriptions of each attribute

In [ ]:
df.isnull()

In [ ]:
#Boxplot to for outliers
for label in cols[:-1]:
    # for label in range(0,5):
    df.plot(
    kind='box', 
    subplots=True, 
    sharey=False, 
    figsize=(70, 25)
    )
        # increase spacing between subplots
    plt.subplots_adjust(wspace=0.9) 
    plt.show()
    break

#Data Cleaning

In [ ]:
selector = VarianceThreshold(0.01) ##Drop unwanted columns using variance as a parameter
dataset1 = selector.fit_transform(df)
scaler = MinMaxScaler() #normalize
dataset2 = scaler.fit_transform(dataset1)

In [ ]:
dataset2

#Split dataset

In [81]:
dataset = pd.DataFrame(dataset2)
train, test = np.split(dataset.sample(frac=1, random_state=42),[int(0.7*len(dataset))])

In [82]:
def scale_dataset(dataframe, oversample = False):
    X = dataframe[dataframe.columns[:-1]].values
    Y = dataframe[dataframe.columns[-1]].values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    if oversample:
        ros = RandomOverSampler()
        X,Y = ros.fit_resample(X,Y)

    data = np.hstack((X,np.reshape(Y,(-1,1))))

    return data, X,Y

In [83]:
train, X_train, Y_train = scale_dataset(train, oversample=True)
test, X_test, Y_test = scale_dataset(test, oversample=False)

In [75]:
train.shape

(3834, 57)

In [76]:
test.shape

(1381, 57)

In [77]:
dataset.shape

(4601, 57)

In [84]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [87]:
knn_model = KNeighborsClassifier(n_neighbors=3)
knn_model.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=3)

In [85]:
KNeighborsClassifier(n_neighbors=3)

KNeighborsClassifier(n_neighbors=3)

In [88]:
Y_pred = knn_model.predict(X_test)

In [89]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.86      0.90       871
         1.0       0.79      0.90      0.84       510

    accuracy                           0.87      1381
   macro avg       0.86      0.88      0.87      1381
weighted avg       0.88      0.87      0.88      1381

